## Photokinetic simulator




In [6]:
#@title Install and Load Packages { display-mode: "form" }

#@markdown Please execute this cell by pressing the _Play_ button 
#@markdown on the left to download and import the required packages 
#@markdown in this Colab notebook.

#@markdown **Note**: This installs the sympy package on the Colab 
#@markdown notebook in the cloud and not on your computer.

try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

try:
    import sympy
except ImportError:
    %shell pip install sympy

from sympy import symbols, Symbol
from IPython.display import display, Math

try:
    import kin_model
except ImportError:
    %shell wget "https://raw.githubusercontent.com/dmadea/Photo-Kinetic-Simulator/main/kin_model.py"

from kin_model import PhotoKineticSymbolicModel

In [7]:
#@title Fontsize of the output cells { run: "auto" }

#@markdown Set the fontsize of the text in output cells,
#@markdown default value is 18px.

font_size =  18 #@param {type:"integer"}

# from https://stackoverflow.com/questions/61957742/how-to-increase-font-size-of-google-colab-cell-output
def increase_font():
    from IPython.display import Javascript
    display(Javascript(f'''
    for (rule of document.styleSheets[0].cssRules){{
        if (rule.selectorText=='body') {{
            rule.style.fontSize = '{font_size}px'
            break
        }}
    }}'''))

if IN_COLAB:
    # set it to run in every cell
    get_ipython().events.register('pre_run_cell', increase_font)
    # disable
    # get_ipython().events.unregister('pre_run_cell', increase_font)

### Input the text model

Use rules as for LaTeX

In [9]:
text_model = """
BR -hv-> ^1BR --> BR          // k_S  # population of singlet state and decay to GS with rate k_S
^1BR --> ^3BR --> BR          // k_{isc} ; k_T
^3BR + ^3O_2 --> ^1O_2 + BR   // k_{TT}
^1O_2 --> ^3O_2               // k_d
^1O_2 + BR --> BR + ^3O_2     // k_q
BR + ^1O_2 -->                // k_r
"""

model = PhotoKineticSymbolicModel.from_text(text_model)
model.print_model()

<IPython.core.display.Math object>

In [10]:
print('\n')
model.pprint_equations()
print('\n')

# substitute for isc quantum yield
k_S, k_isc, *_ = model.symbols['rate_constants']
phi_isc = symbols('\phi_{isc}')
subs = [(k_isc / (k_isc + k_S), phi_isc)]

model.steady_state_approx(['^1BR', '^3BR', '^1O_2'], subs)  # set the species for SS approximation

Eq(Derivative(c_{BR}(t), t), -J + k_S*c_{^1BR}(t) + k_T*c_{^3BR}(t) - k_r*c_{BR}(t)*c_{^1O_2}(t) + k_{TT}*c_{^3BR}(t)*c_{^3O_2}(t))

Eq(Derivative(c_{^1BR}(t), t), J - k_S*c_{^1BR}(t) - k_{isc}*c_{^1BR}(t))

Eq(Derivative(c_{^3BR}(t), t), -k_T*c_{^3BR}(t) - k_{TT}*c_{^3BR}(t)*c_{^3O_2}(t) + k_{isc}*c_{^1BR}(t))

Eq(Derivative(c_{^3O_2}(t), t), k_d*c_{^1O_2}(t) + k_q*c_{BR}(t)*c_{^1O_2}(t) - k_{TT}*c_{^3BR}(t)*c_{^3O_2}(t))

Eq(Derivative(c_{^1O_2}(t), t), -k_d*c_{^1O_2}(t) - k_q*c_{BR}(t)*c_{^1O_2}(t) - k_r*c_{BR}(t)*c_{^1O_2}(t) + k_{TT}*c_{^3BR}(t)*c_{^3O_2}(t))

Eq(Derivative(c_{BR}(t), t), -J*\phi_{isc}*k_r*k_{TT}*c_{BR}(t)*c_{^3O_2}(t)/((k_T + k_{TT}*c_{^3O_2}(t))*(k_d + k_q*c_{BR}(t) + k_r*c_{BR}(t))))

Eq(c_{^1BR}(t), J/(k_S + k_{isc}))

Eq(c_{^3BR}(t), J*\phi_{isc}/(k_T + k_{TT}*c_{^3O_2}(t)))

Eq(Derivative(c_{^3O_2}(t), t), -J*\phi_{isc}*k_r*k_{TT}*c_{BR}(t)*c_{^3O_2}(t)/((k_T + k_{TT}*c_{^3O_2}(t))*(k_d + k_q*c_{BR}(t) + k_r*c_{BR}(t))))

Eq(c_{^1O_2}(t), J*\phi_{isc}*k_{TT}*c_{^3O_2}(t)/((k_T + k_{TT}*c_{^3O_2}(t))*(k_d + k_q*c_{BR}(t) + k_r*c_{BR}(t))))

In [29]:
from sympy import lambdify

J = model.symbols['flux']
k_S, k_isc, k_T, k_TT, k_d, k_q, k_r = model.symbols['rate_constants']
_, c, *_ = model.symbols['compartments']


f = lambdify([J, k_S, k_isc, c], model.symbols['equations'][1].rhs, 'numpy')



# model.symbols['equations'][1].rhs

f(1, 1, 1, 5)

-9

In [5]:
str_model = """
ArO_2 --> Ar + ^1O_2             // k_1  # absorption and singlet state decay
^1O_2 --> ^3O_2                  // k_d
^1O_2 + Ar --> Ar + ^3O_2         // k_{q,Ar}
^1O_2 + ArO_2 --> ArO_2 + ^3O_2     // k_{q,ArO_2}
^1O_2 + S --> S + ^3O_2           // k_{q,S}
S + ^1O_2 -->                      // k_r
"""

model = PhotoKineticSymbolicModel.from_text(str_model)

model.pprint_model()
print('\n')
# model.pprint_equations()
# print('\n')

model.steady_state_approx(['^1O_2'])  # set the species for SS approximation

<IPython.core.display.Math object>

Eq(Derivative(c_{ArO_2}(t), t), -k_1*c_{ArO_2}(t))

Eq(Derivative(c_{Ar}(t), t), k_1*c_{ArO_2}(t))

Eq(c_{^1O_2}(t), k_1*c_{ArO_2}(t)/(k_d + k_r*c_{S}(t) + k_{q,ArO_2}*c_{ArO_2}(t) + k_{q,Ar}*c_{Ar}(t) + k_{q,S}*c_{S}(t)))

Eq(Derivative(c_{^3O_2}(t), t), k_1*(k_d + k_{q,ArO_2}*c_{ArO_2}(t) + k_{q,Ar}*c_{Ar}(t) + k_{q,S}*c_{S}(t))*c_{ArO_2}(t)/(k_d + k_r*c_{S}(t) + k_{q,ArO_2}*c_{ArO_2}(t) + k_{q,Ar}*c_{Ar}(t) + k_{q,S}*c_{S}(t)))

Eq(Derivative(c_{S}(t), t), -k_1*k_r*c_{ArO_2}(t)*c_{S}(t)/(k_d + k_r*c_{S}(t) + k_{q,ArO_2}*c_{ArO_2}(t) + k_{q,Ar}*c_{Ar}(t) + k_{q,S}*c_{S}(t)))